In [ ]:
### PROGRAMA DE SCRAPPING DE INFORMAÇÕES DO METACRITIC VOLTADO PARA JOGOS
### DEVERA PEGAR O NOME DO JOGO, SUA NOTA MEDIA DA CRITICA E DOS USUARIOS
### DATA DE LANÇAMENTO DO JOGO. E ORGANIZA-LOS EM ORDEM DECRESCENTE.

In [ ]:
### CRIANDO POR: VICTOR KRONNER
### DATA DE CRIAÇÃO: 04/03/2022

In [ ]:
### URL: https://www.metacritic.com/browse/games/score/metascore/all/all/filtered

In [ ]:
### FOI UTILIZADO FIREBASE PARA ENVIAR OS DADOS PARA UM BANCO DE DADOS.
### o CODIGO NÃO CONSTA NESTA VERSÃO DEVIDO A ESTES DADOS JA ESTAREM NO
### BANCO SEM NECESSIDADE DE SE REPETIR. SERÁ MANTIDO COMO COMENTADO NO
### CODIGO PARA FUTURA REFERENCIA.

In [41]:
import re
import requests
import pandas as pd
import firebase_admin
#from firebase_admin import db
#from firebase_admin import credentials
from bs4 import BeautifulSoup

In [42]:
page_counter = 0

URL = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered/?page="

df = pd.DataFrame

my_data = {}

thisHeaders = {'User-Agent': 'Chrome/39.0.2171.95'}

lst_names = []
lst_plataform = []
lst_release = []
lst_meta_score = []
lst_user_score = []

plataform = []
games = []
releases = []
meta_scores = []
user_scores = []

In [43]:
"""
cred = credentials.Certificate('firebaseToken.json')

firebase_admin.initialize_app(cred,{
    'databaseURL':'https://scrappermetacritic-default-rtdb.firebaseio.com/'
})
"""

"\ncred = credentials.Certificate('firebaseToken.json')\n\nfirebase_admin.initialize_app(cred,{\n    'databaseURL':'https://scrappermetacritic-default-rtdb.firebaseio.com/'\n})\n"

In [44]:
def URL_Request(myURL, MyPage_counter):
  
  myFinalURL = "{0}{1}".format(URL,MyPage_counter)  
  return myFinalURL

In [45]:
def Get_Game_Names(soup):
  
  for table in soup.find_all("table"):
    for game in table.find_all("h3"):
      games.append(game.text)
  return games

In [46]:
def Get_Plataform_Game(soup):
  
  for table in soup.find_all("td"):
    for info_data in table.find_all("div", {"class": "clamp-details"}):
      for data in info_data.find_all("span", {"class": "data"}):
        aux = data.text
        aux = re.sub(r"\s+","",aux)
        plataform.append(aux)
  return plataform

In [47]:
def Get_Meta_Score(soup):

  for table in soup.find_all("table"):
    for nota_info in table.find_all("div", {"class": "browse-score-clamp"}):
      for nota_meta in nota_info.find_all("div", {"metascore_w large game positive", "metascore_w large game mixed", "metascore_w large game negative", "metascore_w large game tbd"}):
          meta_scores.append(nota_meta.text)
  return meta_scores

In [48]:
def Get_User_Score(soup):

  for table in soup.find_all("table"):
    for nota_info in table.find_all("div", {"class": "browse-score-clamp"}):
      for nota_user in nota_info.find_all("div", {"metascore_w user large game positive", "metascore_w user large game mixed", "metascore_w user large game negative", "metascore_w user large game tbd"}):
          user_scores.append(nota_user.text)
  return user_scores

In [49]:
def Get_Data_From_Game(soup):
  
  for table in soup.find_all("td"):
    for info_data in table.find_all("div", {"class": "clamp-details"}):
      for data in info_data.find_all("span", {"class": ""}):
        releases.append(re.split("\n", data.text))
  return releases

In [50]:
def Start_Scrapping(myURL, myHeaders):

  conteudo_html = requests.get(myURL, headers = myHeaders).text
  soup = BeautifulSoup(conteudo_html, "lxml")

  global lst_names
  global lst_plataform
  global lst_meta_score
  global lst_user_score
  global lst_release

  print(soup.title.text)

  lst_names = Get_Game_Names(soup)
  lst_plataform = Get_Plataform_Game(soup)
  lst_meta_score = Get_Meta_Score(soup)
  lst_user_score = Get_User_Score(soup)
  lst_release = Get_Data_From_Game(soup)
  return

In [51]:
### CREATE A DATA STRUCTURE IN A DATAFRAME TO USE IN PANDAS

def CreatePandaStructure(myGameName, myGamePlataform, myMetaScore, myUserScore, myGameData):
  data = {
          'Name': myGameName,
          'Plataform': myGamePlataform,
          'Meta Score': myMetaScore,
          'User Score': myUserScore,
          'Release': myGameData
          }
  return data

In [ ]:
### PUSH ALL DATA FROM SCRAPPER TO DATABASE FIREBASE

"""
def PushToBD(nomes, plataforrmas, datas, notasMeta, notasUser):
    
    ref = db.reference('/Jogos/')
    for each in range(len(nomes)):
        eachGame = "game{0}".format(each)        
        ref.push({
          'Game_{0}'.format(each + 1): {
              'lancamento':'{0}'.format(datas[each]),
              'nome':'{0}'.format(nomes[each]),
              'nota_meta':'{0}'.format(notasMeta[each]),
              'nota_publico':'{0}'.format(notasUser[each]),
              'plataforma':'{0}'.format(plataforrmas[each])
          }  
        })
    return
"""

In [53]:
def __main__():
  exit = False
  html_content = None

  global page_counter

  page_counter = 0
  page_limiter = 10

  while exit != True:

      html_content = requests.get(URL_Request(URL,page_counter), headers = thisHeaders)

      if html_content.status_code == 200:
        # Code 200 MEANS THAT THE TRANSACTION WAS SUCCESSFUL
        html_content = requests.get(URL_Request(URL,page_counter), headers = thisHeaders)
        Start_Scrapping(URL_Request(URL,page_counter), thisHeaders)
      else:
        # PRINT DENIED IF THE TRANSACTION IS DENIED
        print("DENIED")
        exit = True
        break

      if page_counter == page_limiter - 1:
        #Page Counter Limiter Utilizar um limite pois as paginas sem dados começam a partir da pagina 193
        #Logo um limite para impedir trabalho exagerado desnecessário.
        exit = True
        break

      page_counter += 1

  #PushToBD(lista_nomes, lista_plataformas, lista_datas, lista_notas_meta, lista_notas_usuarios)
  
  df = pd.DataFrame(data = CreatePandaStructure(lst_names, lst_plataform, lst_meta_score, lst_user_score, lst_release))

  return df
  #Imprime os dados em formato de Data Frame podendo ser organizado da maneira que for mais a calhar
  #ao usuario.

In [ ]:
__main__()